# 决策树

## Metrics 
### Gini impurity
应用于CART分类决策树,Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. The Gini impurity can be computed by summing the probability $p_{i}$ of an item with label i being chosen times the probability $\sum _{k\neq i}p_{k}=1-p_{i}$ of a mistake in categorizing that item. It reaches its minimum (zero) when all cases in the node fall into a single target category.
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/e3c72a8e705c2fddf2fd552241c579a6c146af7f"></img>

### Information gain  
Used by the ID3, C4.5 and C5.0 tree-generation algorithms. Information gain is based on the concept of entropy and information content from information theory.   
熵定义:${H} (T)=\operatorname {I} _{E}\left(p_{1},p_{2},...,p_{J}\right)=-\sum _{i=1}^{J}{p_{i}\log _{2}p_{i}}$
上述定义是对离散型随机变量的定义  
where $\displaystyle p_{1},p_{2},...$ $\displaystyle p_{1},p_{2},...$are fractions that add up to 1 and represent the percentage of each class present in the child node that results from a split in the tree.  
$\displaystyle \overbrace {IG(T,a)} ^{\text{Information Gain}}=\overbrace {\mathrm {H} (T)} ^{\text{Entropy (parent)}}-\overbrace {\mathrm {H} (T|a)} ^{\text{Weighted Sum of Entropy (Children)}}$ $\displaystyle =-\sum _{i=1}^{J}p_{i}\log _{2}{p_{i}}-\sum _{a}{p(a)\sum _{i=1}^{J}-\Pr(i|a)\log _{2}{\Pr(i|a)}}$ $\displaystyle =-\sum _{i=1}^{J}p_{i}\log _{2}{p_{i}}-\sum _{a}{p(a)\sum _{i=1}^{J}-\Pr(i|a)\log _{2}{\Pr(i|a)}}$

## ID3  
伪代码,是C4.5的前一个版本
```
ID3 (Examples, Target_Attribute, Attributes)
    Create a root node for the tree
    If all examples are positive, Return the single-node tree Root, with label = +.
    If all examples are negative, Return the single-node tree Root, with label = -.
    If number of predicting attributes is empty, then Return the single node tree Root,
    with label = most common value of the target attribute in the examples.
    Otherwise Begin
        A ← The Attribute that best classifies examples.
        Decision Tree attribute for Root = A.
        For each possible value, vi, of A,
            Add a new tree branch below Root, corresponding to the test A = vi.
            Let Examples(vi) be the subset of examples that have the value vi for A
            If Examples(vi) is empty
                Then below this new branch add a leaf node with label = most common target value in the examples
            Else below this new branch add the subtree ID3 (Examples(vi), Target_Attribute, Attributes – {A})
    End
    Return Root```  
容易过拟合,不支持连续型特征,只能用于分类

### python implementation

In [5]:
! cat ID3.py

import pandas as pd
from scipy.stats import entropy
from collections import Counter, defaultdict
import numpy as np


class ID3:
    def __init__(self):
        self.root = None
        self.features = None

    def fit(self, x: pd.DataFrame, y: pd.Series):
        """
        训练ID3模型
        :param x: pandas.DataFrame 训练特征数据
        :param y: pandas.Series 目标数据
        :return:
        """
        self.features = x.columns
        self.root = ID3._split_data(x, y, x.columns)

    def predict(self, x):
        assert len(x.shape) == 2
        result = []
        for i in range(x.shape[0]):
            result.append(ID3._look_up(x.iloc[i, :], self.root))
        return pd.Series(result)

    @staticmethod
    def _look_up(features, node):
        label = None
        if "label" in node:
            label = node["label"]
        else:
            for n in node["nodes"]:
                fe_name = n["split_feature"]
                if features[fe_name] == n["feature_value"]:
              

## C4.5决策树
C4.5相对于ID3有以下的几个好处:
* 使用了归一化的gain
* 支持数值型特征,使用最大值与最小值的中值作为边界  
目前不支持回归问题

### Python Implementation

In [7]:
! cat C45.py

import pandas as pd
from scipy.stats import entropy
from collections import Counter, defaultdict
import numpy as np


class C45:
    def __init__(self):
        self.root = None

    def fit(self, x: pd.DataFrame, y: pd.Series):
        """
        训练C4.5模型
        :param x: pandas.DataFrame 训练特征数据
        :param y: pandas.Series 目标数据
        :return:
        """
        self.root = C45._split_data(x, y, x.columns)

    def predict(self, x):
        assert len(x.shape) == 2
        result = []
        for i in range(x.shape[0]):
            result.append(C45._look_up(x.iloc[i, :], self.root))
        return pd.Series(result)

    @staticmethod
    def _look_up(features, node):
        label = None
        if node.is_leaf:
            label = node.label
        else:
            for item in node.nodes:
                fe_name = node.split_feature
                if node.threshold is None:
                    if features[fe_name] == item.feature_value:
                        label = C45

## CART回归树
回归树生成过程与C4.5类似,回归树使用数据的均值作为预测值  
通过最小化叶子结点数据的平方差来选择最有划分.  
在选则划分点的时候理论上需要全量搜索所有可能的划分值,然后选择最优划分点,这一步会存在性能问题  
划分结束可以通过阈值来设定和叶子节点的最小数据数量  

## Metrics

决策树划分时使用的度量有:  
分类  
* IG(information gain):信息增益
* gain ratio:标准化的信息增益
* gini impurity:基尼指数,两次抽样,类别不同的概率  

回归  
* MSE  